In [1]:
import pandas as pd

In [8]:
combined_news = pd.read_csv('Combined_News_DJIA.csv')
rnews = pd.read_csv('RedditNews.csv')
stock = pd.read_csv('DJIA_table.csv')


In [10]:
combined_news.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1989 entries, 0 to 1988
Data columns (total 27 columns):
Date     1989 non-null object
Label    1989 non-null int64
Top1     1989 non-null object
Top2     1989 non-null object
Top3     1989 non-null object
Top4     1989 non-null object
Top5     1989 non-null object
Top6     1989 non-null object
Top7     1989 non-null object
Top8     1989 non-null object
Top9     1989 non-null object
Top10    1989 non-null object
Top11    1989 non-null object
Top12    1989 non-null object
Top13    1989 non-null object
Top14    1989 non-null object
Top15    1989 non-null object
Top16    1989 non-null object
Top17    1989 non-null object
Top18    1989 non-null object
Top19    1989 non-null object
Top20    1989 non-null object
Top21    1989 non-null object
Top22    1989 non-null object
Top23    1988 non-null object
Top24    1986 non-null object
Top25    1986 non-null object
dtypes: int64(1), object(26)
memory usage: 419.6+ KB


In [40]:
combined_news = combined_news.set_index('Date')


KeyError: 'Date'

In [41]:
combined_news.isna().sum().sum() #누락체크
combined_news = combined_news.dropna()
sum(combined_news.index <'2014-12-31'), sum(combined_news.index >'2014-12-31')

(1607, 378)

In [42]:
X = combined_news.drop(columns='Label')
y = combined_news['Label']
combined_news.shape, X.shape, y.shape


((1986, 26), (1986, 25), (1986,))

In [47]:
x_flat = X.values.reshape([-1])
x_flat

array(['b"Georgia \'downs two Russian warplanes\' as countries move to brink of war"',
       "b'BREAKING: Musharraf to be impeached.'",
       "b'Russia Today: Columns of troops roll into South Ossetia; footage from fighting (YouTube)'",
       ...,
       'Venezuela, where anger over food shortages is still mounting, continued to be roiled this week by angry protests and break-ins of grocery stores and businesses that have left five dead, at least 30 injured and 200 arrested, according to various news reports.',
       'A Hindu temple worker has been killed by three men on a motorcycle, local police have said. More than 40 people have been killed in attacks blamed on Islamist militants in Bangladesh since February 2013, including secular bloggers, academics, gay rights activists and members of religious minorities.',
       'Ozone layer hole seems to be healing - US &amp; UK team shows it\'s shrunk &amp; may slowly recover. "If you had to have an ozone hole anywhere in the world, it\

In [79]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer # 갯수를 세어서 배열로 만든다.

In [107]:
vect = CountVectorizer()

# 3%이하 / 97% 이상 등장하는 단어는 제외, 최대 어휘 20만 제한, N그램 2(단어 2개씩 자르기)
tfv = TfidfVectorizer( min_df=0.0039, max_df=0.1, max_features = 200000, ngram_range = (3, 3))

In [108]:
tfv = TfidfVectorizer().fit(x_flat)

In [85]:
vect = CountVectorizer().fit(x_flat)

In [50]:
len(vect.vocabulary_)

34682

In [109]:
x_sparse = tfv.transform(x_flat) # 자연어를 벡터로 변환

In [64]:
train_m = 1608
#sum(combined_news.index <= '2014-12-31')

In [63]:
feature_n = X.shape[1]


In [69]:
y_train.shape, x_train.shape
import numpy as np

선형모델, X의 dim 이 2이어야 합니다. 
- 방안 1. 뉴스1~25를 합친다. 
- 방안 2. spares_matrix 25개를 합친다. 


In [102]:
x_agr =[]
for i in range(len(X)) : 
    one_day_news = x_sparse[i*25 : (i+1)*25] #<25*34682>
    new_sum = np.sum(one_day_news,axis=0) #<34682>
    x_agr.append(new_sum)
len(x_agr)

1986

In [103]:
from sklearn.linear_model import RidgeClassifier

In [110]:
x_2d = np.concatenate(x_agr,axis=0)
x_2d.shape, y.shape

((1986, 34682), (1986,))

In [111]:
y_train = y.values[:train_m]
y_test = y.values[train_m:]
x_train = x_2d[:train_m] #
x_test = x_2d[train_m:]

y_train.shape, y_test.shape, x_train.shape, x_test.shape

((1608,), (378,), (1608, 34682), (378, 34682))

In [113]:
model = RidgeClassifier().fit(x_train,y_train)
model.score(x_train,y_train), model.score(x_test,y_test)

(1.0, 0.4497354497354497)

- 주가 데이터(수치형) + 뉴스 데이터(Text > vector) 2~3만개
- 컬럼 / open, close, low, high, low, volume, 그날의 뉴스 벡터
- 주가 데이터 분석 모델 + 뉴스 분석 모델 > 주식 시장 예측


In [114]:
stock.head()

,Date,Open,High,Low,Close,Volume,Adj Close
0,2016-07-01,17924.240234,18002.380859,17916.910156,17949.369141,82160000,17949.369141
1,2016-06-30,17712.759766,17930.609375,17711.800781,17929.990234,133030000,17929.990234
2,2016-06-29,17456.019531,17704.509766,17456.019531,17694.679688,106380000,17694.679688
3,2016-06-28,17190.509766,17409.720703,17190.509766,17409.720703,112190000,17409.720703
4,2016-06-27,17355.210938,17355.210938,17063.080078,17140.240234,138740000,17140.240234


In [149]:
stock = stock.set_index('Date')

In [120]:
rnews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 73608 entries, 0 to 73607
Data columns (total 2 columns):
Date    73608 non-null object
News    73608 non-null object
dtypes: object(2)
memory usage: 1.1+ MB


In [154]:
pd.to_datetime(rnews['Date']), pd.to_datetime(stock['Date'])



KeyError: 'Date'

In [160]:
stock_new = pd.concat((join_axes=1,stock,reddit_agre),axis=1)

SyntaxError: invalid syntax (<ipython-input-160-98ba8a727ed4>, line 1)

In [157]:
stock_new.head()

,Open,High,Low,Close,Volume,Adj Close,News
2008-06-08,NaN,NaN,NaN,NaN,NaN,NaN,b'Nim Chimpsky: The tragedy of the chimp who t...
2008-06-09,NaN,NaN,NaN,NaN,NaN,NaN,"b'United States quits Human Rights Council'b""P..."
2008-06-10,NaN,NaN,NaN,NaN,NaN,NaN,"b'Oil shortage a myth, says industry insider'b..."
2008-06-11,NaN,NaN,NaN,NaN,NaN,NaN,b'BBC uncovers lost Iraq billions'b'War Crimin...
2008-06-12,NaN,NaN,NaN,NaN,NaN,NaN,"b'Did US attack Pakistan last night? ""Pakistan..."


In [133]:
reddit_agre = rnews.groupby('Date').sum() #같은 날짜의 뉴스 붙이기

In [136]:
len(reddit_agre), len(stock)

(2943, 1989)

In [144]:
reddit_agre.isna().sum()
reddit_1d = reddit_agre.values.reshape([-1])

In [139]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer # 갯수를 세어서 배열로 만든다.


In [145]:
vect = CountVectorizer().fit(reddit_1d)

In [146]:
vect.vocabulary_

{'nim': 42246,
 'chimpsky': 12380,
 'the': 62065,
 'tragedy': 63289,
 'of': 43288,
 'chimp': 12375,
 'who': 67949,
 'thought': 62279,
 'he': 28293,
 'was': 67257,
 'boy': 9476,
 'and': 4539,
 'proved': 49219,
 'that': 62049,
 'humans': 29790,
 'were': 67790,
 'not': 42578,
 'humane': 29764,
 'canada': 10762,
 'beware': 8157,
 'slippery': 57233,
 'slope': 57243,
 'to': 62734,
 'censorship': 11624,
 'hearing': 28385,
 'told': 62838,
 'eu': 22011,
 'vice': 66204,
 'president': 48107,
 'luisa': 36979,
 'morgantini': 40236,
 'irish': 32274,
 'nobel': 42393,
 'laureate': 35303,
 'mairead': 37314,
 'corrigan': 15195,
 'have': 28228,
 'been': 7740,
 'tear': 61436,
 'gased': 25824,
 'injured': 31223,
 'by': 10414,
 'idf': 30111,
 'while': 67891,
 'attending': 6248,
 'international': 31675,
 'conference': 14367,
 'on': 43703,
 'non': 42443,
 'violent': 66492,
 'resistance': 52205,
 'israeli': 32646,
 'minister': 39340,
 'israel': 32597,
 'will': 68115,
 'attack': 6042,
 'iran': 32059,
 'if': 301

In [147]:
vect.transform(reddit_1d)

<2943x69741 sparse matrix of type '<class 'numpy.int64'>'
	with 881584 stored elements in Compressed Sparse Row format>

In [151]:
stock.index

Index(['2016-07-01', '2016-06-30', '2016-06-29', '2016-06-28', '2016-06-27',
       '2016-06-24', '2016-06-23', '2016-06-22', '2016-06-21', '2016-06-20',
       ...
       '2008-08-21', '2008-08-20', '2008-08-19', '2008-08-18', '2008-08-15',
       '2008-08-14', '2008-08-13', '2008-08-12', '2008-08-11', '2008-08-08'],
      dtype='object', name='Date', length=1989)

In [153]:
reddit_agre

Index(['2008-06-08', '2008-06-09', '2008-06-10', '2008-06-11', '2008-06-12',
       '2008-06-13', '2008-06-14', '2008-06-15', '2008-06-16', '2008-06-17',
       ...
       '2016-06-22', '2016-06-23', '2016-06-24', '2016-06-25', '2016-06-26',
       '2016-06-27', '2016-06-28', '2016-06-29', '2016-06-30', '2016-07-01'],
      dtype='object', name='Date', length=2943)